# Feature Drift Detection on Real-World Sensor Data

## Overview

In production machine learning systems, models are trained on historical data — but the real world changes over time. **Feature drift** (also called covariate shift) occurs when the statistical distribution of input features changes, causing a model's predictions to silently degrade even when no code has changed.

This notebook implements a lightweight drift detection pipeline and applies it to the [UCI Air Quality dataset](https://archive.ics.uci.edu/dataset/360/air+quality): hourly multisensor readings from an Italian city spanning March 2004 to April 2005.

**Feature monitored:** `C6H6(GT)` — ground-truth benzene concentration (μg/m³), a feature strongly correlated with CO levels and known to exhibit seasonal variation.

**Methods used:**
- **Kolmogorov-Smirnov (KS) two-sample test** — a non-parametric test of whether two samples are drawn from the same distribution
- **Wasserstein distance (Earth Mover's Distance)** — measures how much "work" is needed to transform one distribution into another

Both methods operate on a **sliding window** basis, comparing a recent test window against a preceding baseline window at regular intervals.

## 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Load Dataset

The dataset is fetched directly via the `ucimlrepo` package. It contains 9358 hourly observations across 13 sensor columns. The target variable `y` contains CO concentration readings; the features `X` contain the remaining sensor and reference analyser readings.

> **Dataset:** Air Quality UCI — [https://archive.ics.uci.edu/dataset/360/air+quality](https://archive.ics.uci.edu/dataset/360/air+quality)  
> **Source:** S. De Vito et al., *Sensors and Actuators B: Chemical*, 2008

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
air_quality = fetch_ucirepo(id=360) 
  
# data (as pandas dataframes) 
X = air_quality.data.features 
y = air_quality.data.targets 
  
# metadata 
print(air_quality.metadata) 
  
# variable information 
print(air_quality.variables) 

## 3. Data Cleaning & Preprocessing

### 3.1 Check for Null Values

In [ ]:
X.isna().any()

### 3.2 Concatenate Date and Time into a Datetime Index

The raw dataset stores date and time as separate string columns using `/` as a date separator. These are combined and parsed into a proper `datetime` index to allow time-based indexing and resampling.

In [ ]:
a = X.Date + ' ' + X.Time
a = a.transform(lambda x: x.replace('/', '-'))
X['Datetime'] = pd.to_datetime(a)

### 3.3 Set Datetime as Index

In [ ]:
X.drop(columns=['Date', 'Time'], inplace=True)
X.set_index(['Datetime'], inplace=True)
X.head()

In [ ]:
X.describe()

### 3.4 Handle Sentinel Missing Values

The dataset encodes missing sensor readings as `-200` rather than `NaN`. These are replaced with `NaN` and the affected rows are dropped. This is standard practice for this dataset and documented in the UCI repository.

In [ ]:
feature_col = 'C6H6(GT)'
X[feature_col] = X[feature_col].replace(-200, np.nan)
X.dropna(subset=[feature_col], inplace=True)

## 4. Exploratory Data Analysis

### 4.1 Seasonal Trend in Benzene Concentration

Based on the rolling average, C6H6 concentration is higher in winter months compared to summer months. This is consistent with atmospheric chemistry: colder temperatures reduce boundary layer mixing, and higher combustion activity (heating) increases benzene emissions. This seasonal pattern constitutes a genuine, real-world example of **covariate shift** — the input distribution changes over time even though the underlying physical process is stable.

In [ ]:
# compute concentration mean over each month and assign it to last day of month
c = X[feature_col].groupby(pd.Grouper(freq='ME')).mean()[1:-1]

plt.figure(figsize=(20,6))
plt.plot(X[feature_col].rolling(24*30).mean(), color='red', label='30-day rolling mean')
plt.scatter(c.index, c, alpha=0.3, label='concentration mean over each month')
plt.plot(c.index, c)
plt.grid(alpha=0.3)
plt.xlabel('time')
plt.ylabel('microg/m^3')
plt.title('C6H6 Concentration')
plt.legend()
plt.show()

### 4.2 Distribution Shift: First 3 Months vs Last 3 Months

Comparing the early period (March–April 2004) against the late period (January–April 2005) makes the distributional difference concrete. The two histograms show a clear shift in the central tendency and shape of the benzene distribution — exactly the kind of drift a deployed model would encounter silently over time.

In [ ]:
a1 = X[feature_col][(X.index.year==2004) & (X.index.month<=4)]
a2 = X[feature_col][(X.index.year==2005) & (X.index.month>=1)]

sns.histplot(a1, alpha=0.4, label='first 3 months')
sns.histplot(a2, alpha=0.4, label='last 3 months')
plt.title('First 3 Vs. Last 3 Months C6H6 Concentration Distribution')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Drift Detection Pipeline

### 5.1 Parameter Setup

The pipeline uses a **sliding window** approach over the ordered time series:

| Parameter | Value | Rationale |
|---|---|---|
| `intervals` | 50 obs | ~2 days of hourly data; granular enough to catch transitions |
| `lookback_to_test_size_ratio` | 3 | Baseline window = 3× test window (150 obs ≈ 6 days) — large enough to be statistically stable |
| `start_point` | 200 obs | Burn-in period: ensures a full baseline window exists before the first test |
| `alpha` (KS test) | 1e-8 | Conservative threshold to minimise false positives given the large sample sizes |

At each test point `t`:
- **Test window (`dist_2`):** observations in `[t - 50, t)`
- **Baseline window (`dist_1`):** observations in `[t - 200, t - 50)` (150 observations)

In [ ]:
data_len = len(X[feature_col])

# performing hypothesis testing after every n interval
intervals = 50
# size compared to interval to consider as population distribution
lookback_to_test_size_ratio = 3
# index to start peforming hypothesis testing
start_point = intervals * (lookback_to_test_size_ratio+1)
# considered alpha for comparing against p-value
a = 1e-8


time_line_tot = X.index
time_line_chosen = X[start_point::intervals]

### 5.2 Run KS Test and Wasserstein Distance

Both metrics are computed in a single loop to avoid redundant window slicing:

- **KS two-sample test (`ks_2samp`):** Tests the null hypothesis H₀ that `dist_1` and `dist_2` are drawn from the same continuous distribution. A very small p-value (< α) leads to rejection of H₀ and flags a potential drift event.

- **Wasserstein distance:** Measures the minimum "cost" of transforming one empirical distribution into the other. Unlike the KS test, it provides a continuous magnitude of shift rather than a binary flag, making it useful for tracking how severe drift is over time.

In [ ]:
p_values_list = []
wasserstein_distance_list = pd.DataFrame(index=time_line_chosen.index, data=[np.nan]*len(time_line_chosen))

for t in time_line_chosen.index:
    current_idx = np.argwhere(X.index == t)[0][0]
    dist_1 = X[feature_col].iloc[current_idx - intervals * (1 + lookback_to_test_size_ratio):current_idx - intervals]
    dist_2 = X[feature_col].iloc[current_idx - intervals:current_idx]
    
    # KS test result
    result = stats.ks_2samp(dist_1, dist_2)
    p_values_list.append(result.pvalue.item())

    # Wasserstein distance result
    distance = stats.wasserstein_distance(dist_1, dist_2)
    wasserstein_distance_list.loc[t] = distance.item()

## 6. Results

### 6.1 KS Test — H₀ Rejection Points

Test points where the p-value falls below `α = 1e-8` are considered drift alerts. These are timestamps where the recent 50-observation window is statistically inconsistent with the preceding 150-observation baseline at a very high confidence level.

In [ ]:
p_value_low = np.array(p_values_list) < a

shift_times = []

for k, v in zip(list(time_line_chosen.index), p_value_low):
  if v == True:
    print(f"Timeline: {k}\t H_0 rejected: {v}")
    shift_times.append(k)

The vertical lines below mark each timestamp where H₀ was rejected. These cluster around the summer trough and the autumn recovery in benzene levels — the two most rapid distributional transitions in the year.

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(X[feature_col].rolling(24*30).mean(), color='red', label='monthly rolling mean')
for t in shift_times:
    plt.axvline(t, color='skyblue', linestyle='--')

plt.grid(alpha=0.3)
plt.xlabel('time')
plt.ylabel('μg/m³')
plt.title('$H_{0}$ Rejection Points Via KS Test')
plt.show()

### 6.2 Wasserstein Distance — Distribution of Values

Before applying a threshold, it is useful to inspect the distribution of all computed Wasserstein distances. This informs a principled, data-driven choice of threshold rather than an arbitrary hardcoded value.

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(wasserstein_distance_list, bins=30, legend=False)
plt.xlabel('Wasserstein distance')
plt.title('Wasserstein distance distribution for C6H6')
plt.grid(alpha=0.3)
plt.tight_layout()

### 6.3 Wasserstein Threshold — 99th Percentile

The alert threshold is set at the **99th percentile** of all observed Wasserstein distances. This means only the top 1% of window pairs — those with the most extreme distributional separation — are flagged. This is a statistically principled approach: the threshold adapts to the actual scale of variation in the data rather than relying on a domain-specific constant.

In [ ]:
threshold_percentile = 99
wasserstein_threshold = np.percentile(wasserstein_distance_list, q=threshold_percentile)

### 6.4 Wasserstein Distance Over Time

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(wasserstein_distance_list)
plt.scatter(wasserstein_distance_list.index, wasserstein_distance_list[0], alpha=0.3)
plt.axhline(wasserstein_threshold, color='red', linestyle='--', alpha=0.3, label='99 percentile threshold')
plt.grid(alpha=0.3)
plt.ylabel('Wasserstein distance')
plt.xlabel('time')
plt.title(f'Wasserstein Distance over Rolling Window of {intervals}')
plt.legend(loc='upper left')
plt.show()

In [ ]:
wasserstein_distance_high = wasserstein_distance_list[wasserstein_distance_list[0] > wasserstein_threshold].index
wasserstein_distance_high

### 6.5 Wasserstein Drift Alert Points on Time Series

The flagged timestamps (those exceeding the 99th percentile threshold) are overlaid on the rolling mean. These align with the same seasonal transitions identified by the KS test, providing **independent corroboration** from a different statistical measure — strengthening confidence that the alerts reflect genuine distributional changes rather than noise.

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(X[feature_col].rolling(24*30).mean(), color='orange', label='monthly rolling mean')
for t in wasserstein_distance_high:
    plt.axvline(t, color='crimson', linestyle='--')

plt.grid(alpha=0.3)
plt.xlabel('time')
plt.ylabel('μg/m³')
plt.title(f'{threshold_percentile}th Percentile Wasserstein Distance Points')
plt.show()

## 7. Summary

The pipeline successfully detected real distributional drift in hourly benzene sensor data, with both methods independently flagging the same seasonal transition periods.

**Method comparison:**

| | KS Test | Wasserstein Distance |
|---|---|---|
| **Output** | p-value (binary flag at threshold α) | Continuous distance score |
| **Sensitivity** | Mean + shape changes | Full distribution shape |
| **Threshold** | Fixed α = 1e-8 | Data-driven (99th percentile) |
| **Strength** | Strong statistical grounding | Magnitude-aware, interpretable |

**Key finding:** Both methods converge on the same drift events, corresponding to the spring–summer drop and autumn rise in benzene concentration — changes driven by real seasonal atmospheric dynamics.

**Limitations and extensions:**
- The current pipeline processes data post-hoc (batch). For online/streaming use, the window buffer would need to be maintained incrementally.
- Only a single feature is monitored. A production system would apply this per feature and aggregate drift scores.
- The KS test assumes independent observations; autocorrelated time series data (as here) can inflate Type I error rates.